In [1]:
print("ok")

ok


In [2]:
%pwd

'd:\\Computer Engineering\\8 th semester\\final year project\\RAG\\MedicalChatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\Computer Engineering\\8 th semester\\final year project\\RAG\\MedicalChatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\Amantha\.conda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob = "*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_files("data")

In [8]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-10-08T20:31:30+05:30', 'author': 'Yasindu Amantha', 'moddate': '2025-10-08T20:31:30+05:30', 'source': 'data\\railway.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='railway_fraud_rules = [ \n \n    # ⏱ Transaction Completion Time-Based Rules \n    "Rule 1: Flag transactions where completion time is between 15 to 4 minutes. Reason: Typical human \ninteraction duration for a genuine booking. Action: Set Fraudulent Risk to Low. Category: Transaction \nCompletion Time-Based Rules", \n    "Rule 2: Flag transactions where completion time is between 0 to 4 minutes. Reason: Extremely fast \nbooking completion—potential automated bot activity. Action: Set Fraudulent Risk to High. Category: \nTransaction Completion Time-Based Rules", \n    "Rule 3: Flag users whose average transaction completion time over the last 28 days is between 15 to \n4 minutes. Reason: Cons

In [9]:
# pages that read
len(extracted_data)

3

In [10]:
# filetr souce and the page content
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of document objects, return a new list of document objects 
    containing only 'source' in metadata and the original page content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [11]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [12]:
minimal_docs

[Document(metadata={'source': 'data\\railway.pdf'}, page_content='railway_fraud_rules = [ \n \n    # ⏱ Transaction Completion Time-Based Rules \n    "Rule 1: Flag transactions where completion time is between 15 to 4 minutes. Reason: Typical human \ninteraction duration for a genuine booking. Action: Set Fraudulent Risk to Low. Category: Transaction \nCompletion Time-Based Rules", \n    "Rule 2: Flag transactions where completion time is between 0 to 4 minutes. Reason: Extremely fast \nbooking completion—potential automated bot activity. Action: Set Fraudulent Risk to High. Category: \nTransaction Completion Time-Based Rules", \n    "Rule 3: Flag users whose average transaction completion time over the last 28 days is between 15 to \n4 minutes. Reason: Consistent human-like behavior. Action: Set Fraudulent Risk to Low. Category: \nTransaction Completion Time-Based Rules", \n    "Rule 4: Flag users whose average transaction completion time over the last 28 days is between 4 to 0 \nminut

In [36]:
# Split the document into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [37]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 24


In [38]:
texts_chunk

[Document(metadata={'source': 'data\\railway.pdf'}, page_content='railway_fraud_rules = [ \n \n    # ⏱ Transaction Completion Time-Based Rules \n    "Rule 1: Flag transactions where completion time is between 15 to 4 minutes. Reason: Typical human \ninteraction duration for a genuine booking. Action: Set Fraudulent Risk to Low. Category: Transaction'),
 Document(metadata={'source': 'data\\railway.pdf'}, page_content='Completion Time-Based Rules", \n    "Rule 2: Flag transactions where completion time is between 0 to 4 minutes. Reason: Extremely fast \nbooking completion—potential automated bot activity. Action: Set Fraudulent Risk to High. Category: \nTransaction Completion Time-Based Rules",'),
 Document(metadata={'source': 'data\\railway.pdf'}, page_content='Transaction Completion Time-Based Rules", \n    "Rule 3: Flag users whose average transaction completion time over the last 28 days is between 15 to \n4 minutes. Reason: Consistent human-like behavior. Action: Set Fraudulent Risk

In [18]:
# Rules

railway_fraud_rules = [
    # ⏱ Transaction Completion Time-Based Rules
    {
        "id": "rule_1",
        "text": "Flag transactions where completion time is between 4 to 15 minutes. Reason: Typical human interaction duration for a genuine booking. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 1,
            "category": "Transaction Completion Time-Based Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "completion_time",
            "threshold_min": 4,
            "threshold_max": 15,
            "unit": "minutes"
        }
    },
    {
        "id": "rule_2",
        "text": "Flag transactions where completion time is between 0 to 4 minutes. Reason: Extremely fast booking completion—potential automated bot activity. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 2,
            "category": "Transaction Completion Time-Based Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "completion_time",
            "threshold_min": 0,
            "threshold_max": 4,
            "unit": "minutes"
        }
    },
    {
        "id": "rule_3",
        "text": "Flag users whose average transaction completion time over the last 28 days is between 4 to 15 minutes. Reason: Consistent human-like behavior. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 3,
            "category": "Transaction Completion Time-Based Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "avg_completion_time_28d",
            "threshold_min": 4,
            "threshold_max": 15,
            "unit": "minutes",
            "time_window": "28_days"
        }
    },
    {
        "id": "rule_4",
        "text": "Flag users whose average transaction completion time over the last 28 days is between 0 to 4 minutes. Reason: Consistently rapid transactions—possible automated booking. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 4,
            "category": "Transaction Completion Time-Based Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "avg_completion_time_28d",
            "threshold_min": 0,
            "threshold_max": 4,
            "unit": "minutes",
            "time_window": "28_days"
        }
    },
    
    # 🌐 IP Address-Based Rules
    {
        "id": "rule_5",
        "text": "Flag users with 3 or fewer unique IP addresses used within the last 28 days. Reason: Normal behavior—same IP or few devices used for booking. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 5,
            "category": "IP Address-Based Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "unique_ip_count_28d",
            "threshold_max": 3,
            "time_window": "28_days"
        }
    },
    {
        "id": "rule_6",
        "text": "Flag users with more than 3 unique IP addresses used within the last 28 days. Reason: Multiple device or proxy usage may indicate account sharing or automation. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 6,
            "category": "IP Address-Based Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "unique_ip_count_28d",
            "threshold_min": 3,
            "time_window": "28_days"
        }
    },
    
    # 📅 Transaction Frequency / Recency Rules
    {
        "id": "rule_7",
        "text": "Flag users whose time since last transaction is more than 3 or 4 days. Reason: Normal gap between bookings—typical travel planning pattern. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 7,
            "category": "Transaction Frequency-Based Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "days_since_last_transaction",
            "threshold_min": 3,
            "unit": "days"
        }
    },
    {
        "id": "rule_8",
        "text": "Flag users whose time since last transaction is less than 3 or 4 days. Reason: High booking frequency may indicate automated ticket purchase attempts. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 8,
            "category": "Transaction Frequency-Based Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "days_since_last_transaction",
            "threshold_max": 3,
            "unit": "days"
        }
    },
    {
        "id": "rule_9",
        "text": "Flag users with no previous transactions (time since last transaction = -1). Reason: New user—no prior suspicious history. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 9,
            "category": "Transaction Frequency-Based Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "days_since_last_transaction",
            "special_value": -1,
            "user_type": "new_user"
        }
    },
    
    # 📆 Weekly Transaction Behavior Rules
    {
        "id": "rule_10",
        "text": "Flag users whose average transactions per week range between 0.22 and 2.04. Reason: Normal ticket purchase frequency for genuine travelers. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 10,
            "category": "Weekly Transaction Behavior Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "avg_transactions_per_week",
            "threshold_min": 0.22,
            "threshold_max": 2.04
        }
    },
    {
        "id": "rule_11",
        "text": "Flag users whose average transactions per week range between 0.3 and 9. Reason: Abnormally high weekly activity—potential bulk or automated purchases. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 11,
            "category": "Weekly Transaction Behavior Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "avg_transactions_per_week",
            "threshold_min": 0.3,
            "threshold_max": 9
        }
    },
    
    # 👥 Dependent Ticket Behavior Rules
    {
        "id": "rule_12",
        "text": "Flag transactions with 0 to 3 dependent tickets. Reason: Typical number of dependent passengers for genuine users. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 12,
            "category": "Dependent Ticket Behavior Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "dependent_ticket_count",
            "threshold_min": 0,
            "threshold_max": 3
        }
    },
    {
        "id": "rule_13",
        "text": "Flag transactions with 3 to 4 dependent tickets. Reason: Higher dependent count may indicate ticket reselling behavior. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 13,
            "category": "Dependent Ticket Behavior Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "dependent_ticket_count",
            "threshold_min": 3,
            "threshold_max": 4
        }
    },
    {
        "id": "rule_14",
        "text": "Flag users whose average dependent transaction ratio over the last 28 days ranges between 0 and 0.41. Reason: Balanced, human-like dependent booking behavior. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 14,
            "category": "Dependent Ticket Behavior Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "avg_dependent_ratio_28d",
            "threshold_min": 0,
            "threshold_max": 0.41,
            "time_window": "28_days"
        }
    },
    {
        "id": "rule_15",
        "text": "Flag users whose average dependent transaction ratio over the last 28 days ranges between 0.5 and 1. Reason: Excessive dependent usage—possible fraudulent bulk booking. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 15,
            "category": "Dependent Ticket Behavior Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "avg_dependent_ratio_28d",
            "threshold_min": 0.5,
            "threshold_max": 1,
            "time_window": "28_days"
        }
    },
    
    # 🎟 Ticket Count-Based Rules
    {
        "id": "rule_16",
        "text": "Flag transactions where number of tickets per transaction ranges between 1 and 3 or 4. Reason: Normal passenger booking behavior. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 16,
            "category": "Ticket Count-Based Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "tickets_per_transaction",
            "threshold_min": 1,
            "threshold_max": 4
        }
    },
    {
        "id": "rule_17",
        "text": "Flag transactions with 4 or 5 tickets. Reason: High-volume purchase per transaction—possible fraud or resale activity. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 17,
            "category": "Ticket Count-Based Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "tickets_per_transaction",
            "threshold_min": 4,
            "threshold_max": 5
        }
    },
    {
        "id": "rule_18",
        "text": "Flag users with an average of 1 to 3 tickets per transaction. Reason: Normal booking size for genuine users. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 18,
            "category": "Ticket Count-Based Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "avg_tickets_per_transaction",
            "threshold_min": 1,
            "threshold_max": 3
        }
    },
    {
        "id": "rule_19",
        "text": "Flag users with an average of 4 or 5 tickets per transaction. Reason: Consistently high ticket count per booking—possible automated group booking or ticket hoarding. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 19,
            "category": "Ticket Count-Based Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "avg_tickets_per_transaction",
            "threshold_min": 4,
            "threshold_max": 5
        }
    },
    
    # 🚆 Ticket Class Usage Rules
    {
        "id": "rule_20",
        "text": "Flag users who buy tickets across multiple classes (e.g., Class 1, 2, 3, Observation Saloon). Reason: Diverse travel patterns—normal behavior. Action: Classify as Genuine (Low Fraud Risk).",
        "metadata": {
            "rule_number": 20,
            "category": "Ticket Class Usage Rules",
            "risk_level": "Low",
            "classification": "Genuine",
            "metric": "ticket_class_diversity",
            "pattern": "multiple_classes"
        }
    },
    {
        "id": "rule_21",
        "text": "Flag users who buy tickets only in Class 1 or Observation Saloon. Reason: Narrow purchase pattern targeting premium classes—possible reseller activity. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 21,
            "category": "Ticket Class Usage Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "ticket_class_diversity",
            "pattern": "premium_only",
            "classes": ["Class 1", "Observation Saloon"]
        }
    },
    
    # 🗺 Route Pattern-Based Rules
    {
        "id": "rule_22",
        "text": "Flag users who consistently book only 'Kandy → Colombo' or 'Colombo → Kandy' routes. Reason: Repeated booking on popular routes—possible automated bulk booking attempt. Action: Classify as Fraudulent (High Fraud Risk).",
        "metadata": {
            "rule_number": 22,
            "category": "Route Pattern-Based Rules",
            "risk_level": "High",
            "classification": "Fraudulent",
            "metric": "route_diversity",
            "pattern": "single_route_repeated",
            "routes": ["Kandy → Colombo", "Colombo → Kandy"]
        }
    }
]

In [2]:
# embedding model
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the hugging face embedding model
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name = model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\Amantha\AppData\Local\Temp\ipykernel_599068\1951281138.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\Amantha\.conda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
# check model is working
vector = embedding.embed_query("Hello world")
vector

[-0.034477200359106064,
 0.031023219227790833,
 0.0067349993623793125,
 0.02610897459089756,
 -0.03936201333999634,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441470701247454,
 -0.04745051637291908,
 0.014758859761059284,
 0.07087533175945282,
 0.05552757531404495,
 0.019193289801478386,
 -0.026251329109072685,
 -0.010109508410096169,
 -0.026940548792481422,
 0.022307483479380608,
 -0.022226618602871895,
 -0.14969265460968018,
 -0.01749304309487343,
 0.007676269859075546,
 0.05435226485133171,
 0.003254482988268137,
 0.03172600269317627,
 -0.0846213772892952,
 -0.02940598875284195,
 0.051595672965049744,
 0.04812409356236458,
 -0.003314794274047017,
 -0.05827922374010086,
 0.041969284415245056,
 0.022210709750652313,
 0.1281888633966446,
 -0.022338956594467163,
 -0.011656241491436958,
 0.06292833387851715,
 -0.032876305282115936,
 -0.09122610837221146,
 -0.03117538057267666,
 0.05269957706332207,
 0.047034792602062225,
 -0.08420304954051971,
 -0.0300561785697937,
 -0.020744718

In [13]:
print("vector length:", len(vector))

vector length: 384


In [3]:
# load .env file
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

os.environ["PINECOEN_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [5]:
# import pinecone
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

# authenticate pinecone account
pc = Pinecone(api_key=pinecone_api_key)

In [7]:
pc

In [6]:
# create database in pinecone
from pinecone import ServerlessSpec

index_name = "railway-rules-new"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension = 384, # dimension of the embedding
        metric = "cosine",
        spec = ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name) 


In [23]:
# ===========================
# 5. INSERT RULES INTO PINECONE
# ===========================
def insert_rules_to_pinecone(rules, index, embedding_model):
    """
    Insert fraud detection rules into Pinecone with embeddings
    """
    vectors_to_upsert = []
    
    for rule in rules:
        # Generate embedding for the rule text
        rule_embedding = embedding_model.embed_query(rule["text"])
        
        # Prepare metadata (Pinecone only supports simple types in metadata)
        metadata = {
            "text": rule["text"],
            "rule_number": rule["metadata"]["rule_number"],
            "category": rule["metadata"]["category"],
            "risk_level": rule["metadata"]["risk_level"],
            "classification": rule["metadata"]["classification"],
            "metric": rule["metadata"]["metric"]
        }
        
        # Add optional fields if they exist
        if "threshold_min" in rule["metadata"]:
            metadata["threshold_min"] = float(rule["metadata"]["threshold_min"])
        if "threshold_max" in rule["metadata"]:
            metadata["threshold_max"] = float(rule["metadata"]["threshold_max"])
        if "unit" in rule["metadata"]:
            metadata["unit"] = rule["metadata"]["unit"]
        if "time_window" in rule["metadata"]:
            metadata["time_window"] = rule["metadata"]["time_window"]
        if "pattern" in rule["metadata"]:
            metadata["pattern"] = rule["metadata"]["pattern"]
        if "user_type" in rule["metadata"]:
            metadata["user_type"] = rule["metadata"]["user_type"]
        if "special_value" in rule["metadata"]:
            metadata["special_value"] = int(rule["metadata"]["special_value"])
        
        # Prepare vector for upsert
        vectors_to_upsert.append({
            "id": rule["id"],
            "values": rule_embedding,
            "metadata": metadata
        })
    
    # Upsert vectors in batches (Pinecone recommends batch size of 100)
    batch_size = 100
    for i in range(0, len(vectors_to_upsert), batch_size):
        batch = vectors_to_upsert[i:i + batch_size]
        index.upsert(vectors=batch)
        print(f"Inserted batch {i//batch_size + 1}: {len(batch)} rules")
    
    print(f"\n✅ Successfully inserted {len(rules)} rules into Pinecone!")
    
    # Get index stats
    stats = index.describe_index_stats()
    print(f"📊 Index stats: {stats}")

In [24]:
# Insert all rules
insert_rules_to_pinecone(railway_fraud_rules, index, embedding)

Inserted batch 1: 22 rules

✅ Successfully inserted 22 rules into Pinecone!
📊 Index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [20]:
# create embeddings for our pdf chunks

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = texts_chunk,
    embedding = embedding,
    index_name = index_name
)

NameError: name 'texts_chunk' is not defined

In [7]:
# Load existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embedding
)

### Add more data to the existing pinecone index

In [4]:
# create dummy document
dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "youtube"}
)

In [18]:
# store too existing pinecone index
docsearch.add_documents(documents=[dswith])

['66e3a354-e572-4197-8e97-b8c89ca2a55d']

Now pinecone vector DB is ready.

### Now connect it with LLM

In [8]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":7})

In [9]:
# ===========================
# TEST 1: Basic Retriever Test
# ===========================
print("=" * 60)
print("TEST 1: Basic Retriever Test")
print("=" * 60)

test_query = "User completed booking in 2 minutes"
retrieved_docs = retriever.get_relevant_documents(test_query)

print(f"\nQuery: '{test_query}'")
print(f"\nRetrieved {len(retrieved_docs)} documents:\n")

for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n{'='*50}")
    print(f"Result {i}:")
    print(f"{'='*50}")
    print(f"Content: {doc.page_content}")
    print(f"\nMetadata:")
    for key, value in doc.metadata.items():
        print(f"  - {key}: {value}")

TEST 1: Basic Retriever Test


C:\Users\Amantha\AppData\Local\Temp\ipykernel_599068\3527517262.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(test_query)



Query: 'User completed booking in 2 minutes'

Retrieved 7 documents:


Result 1:
Content: Flag users whose average transaction completion time over the last 28 days is between 0 to 4 minutes. Reason: Consistently rapid transactions—possible automated booking. Action: Classify as Fraudulent (High Fraud Risk).

Metadata:
  - category: Transaction Completion Time-Based Rules
  - classification: Fraudulent
  - metric: avg_completion_time_28d
  - risk_level: High
  - rule_number: 4.0
  - threshold_max: 4.0
  - threshold_min: 0.0
  - time_window: 28_days
  - unit: minutes

Result 2:
Content: Flag transactions where completion time is between 4 to 15 minutes. Reason: Typical human interaction duration for a genuine booking. Action: Classify as Genuine (Low Fraud Risk).

Metadata:
  - category: Transaction Completion Time-Based Rules
  - classification: Genuine
  - metric: completion_time
  - risk_level: Low
  - rule_number: 1.0
  - threshold_max: 15.0
  - threshold_min: 4.0
  - unit: minutes

In [20]:
retrieved_docs = retriever.invoke("what is Acne?")
retrieved_docs

[Document(id='4b7f46aa-46a3-4d24-a4c0-fbe51e3e2a00', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='1e9f42af-c161-492d-a538-65d160bb5395', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='7b8a2808-2418-44b2-8845-7e925158aeff', metadata={'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged wi

ABove response is not understanding.
Then connect the response with LLM to get a meaningful answer.

### Connect with LLM

In [37]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

generator = pipeline("text2text-generation", model="google/flan-t5-base")
llm = HuggingFacePipeline(pipeline=generator)

c:\Users\Amantha\.conda\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Amantha\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP 

In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following prices of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

NameError: name 'ChatPromptTemplate' is not defined

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

NameError: name 'create_stuff_documents_chain' is not defined

In [39]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder in which the abnormal release of a particular chemical from the pituitary gland in the brain causes increased growth in bone and soft tis- sue, as well as a variety of other disturbances throughout the body.


In [40]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

a skin disorder in which the sebaceous glands become inflamed


In [41]:
response = rag_chain.invoke({"input": "what is the Treatment of Acne?"})
print(response["answer"])

reducing the formation of new comedones with topical tretinoin, benzoyl peroxide, ada- palene, or salicylic acid
